In [2]:
import numpy as np
import horus as ho
# import pyoculus
# from mayavi import mlab
# from pyoculus.problems import CartesianBfield
from pyoculus.solvers import PoincarePlot
from pyoculus.solvers import FixedPoint
from simsopt.field import MagneticField

In [3]:
bs, bsh, (nfp, coils, ma, _) = ho.ncsx()

Setting up a problem

In [4]:
ps = ho.SimsoptBfieldProblem(ma.gamma()[0, 0], 0, 3, bs)

#### Poincare plot 
as in https://github.com/zhisong/pyoculus/blob/master/examples/two_waves.ipynb

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-9

# set up the Poincare plot
pparams = dict()
pparams["Rbegin"] = 1.0
pparams["Rend"] = 1.7
pparams["nPtrj"] = 10
pparams["nPpts"] = 100
pparams["zeta"] = 0.33 * np.pi

pplot = PoincarePlot(ps, pparams, integrator_params=iparams)
pdata = pplot.compute()
pplot.plot()

In [ ]:
pplot.compute_iota()
pplot.plot_iota()

#### Finding fixed point

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 100

fp01 = FixedPoint(ps, pparams, integrator_params=iparams)
result01 = fp01.compute(guess=[1.527, 0.0], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-8)

In [ ]:
# showing the resutlts
if result01 is not None:
    results = [list(p) for p in zip(result01.x, result01.y, result01.z)]
    print(results)

In [ ]:
result01.GreenesResidue

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
# pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 10000

fp01 = FixedPoint(ps, pparams, integrator_params=iparams)
result01 = fp01.compute(guess=[1.527, -0.1], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-12, thetaf = False)

# showing the resutlts
if result01 is not None:
    results = [list(p) for p in zip(result01.x, result01.y, result01.z)]
    print(results)

We can check at the comet by hand

In [ ]:
ma.gamma()[0]

In [ ]:
phi = 0.33*np.pi
R = 1.15
startpoint = [R*np.cos(phi), R*np.sin(phi), 0]
startpoint = [1.9383899348191092, 0.0, 0.0]

In [ ]:
ma_gamma = ho.trace(bs, 8*2*np.pi, ma.gamma()[0])
# gamma = ho.trace(bs, 20*np.pi, startpoint)
#gamma2 = ho.trace(bs, 3*np.pi, startpoint)

In [ ]:
# mask = np.ones(gamma.shape[1], dtype=bool)
# for i, x in enumerate(gamma.T):
#     mask[i] = 0
#     if np.isin(gamma, gamma.T[mask]).all():
#         print(i)
#         break
#     mask[i] = 1

In [ ]:
#gammaX = ho.trace(bs, 21*np.pi, results[0])
gammaO = ho.trace(bs, 200*np.pi, [1.6282982137472521, 0.0, -0.5360026587795327])

In [ ]:
from mayavi import mlab
for coil in coils:
    coil.plot(engine="mayavi", show=False)

#mlab.plot3d(gammaX[0, :], gammaX[1, :], gammaX[2, :], tube_radius=0.01, color=(0, 0, 0))
mlab.plot3d(gammaO[0, :], gammaO[1, :], gammaO[2, :], tube_radius=0.01, color=(0, 0, 1))
#mlab.plot3d(ma_gamma[0, :], ma_gamma[1, :], ma_gamma[2, :], tube_radius=0.01, color=(1, 0, 0))

# Save the VTK object to a file
mlab.show()

#### Map of convergence

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
# pparams["Z"] = 0
pparams["tol"] = 1e-7
pparams['niter'] = 100

fp = FixedPoint(ps, pparams, integrator_params=iparams)

In [ ]:
R,Z = np.meshgrid(np.linspace(1.25, 1.8, 20), np.linspace(-0.6, 0.6, 20))

In [ ]:
assigned_to = list()
fixed_points = list()

In [ ]:
for i, (r, z) in enumerate(zip(R.flatten(), Z.flatten())):
    fp_result = fp.compute(guess=[r, z], pp=3, qq=7, sbegin=1.5, send=1.9, tol = 1e-8)

    if fp_result is not None:
        fp_result_xyz = np.array([fp_result.x[0], fp_result.y[0], fp_result.z[0]])
        assigned = False
        for j, fpt in enumerate(fixed_points):
            fpt_xyz = np.array([fpt.x[0], fpt.y[0], fpt.z[0]])
            if np.isclose(fp_result_xyz, fpt_xyz, atol=1e-3).all():
                assigned_to.append(j)
                assigned = True
        if not assigned:
            assigned_to.append(len(fixed_points))        
            fixed_points.append(fp_result)
    else:
        assigned_to.append(-1)

In [ ]:
assigned_to = np.array(assigned_to)+1

In [ ]:
from matplotlib.colors import TABLEAU_COLORS as colors
colors = list(colors.values())

cmap = np.array([list(int(colors[j].lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) for j in assigned_to])/255

In [ ]:
cmap = cmap.reshape(R.shape[0], R.shape[1], 3)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.pcolormesh(R, Z, cmap, shading='nearest')

for i, fpt in enumerate(fixed_points):
    if fpt.GreenesResidue < 0:
        marker = 'X'
    elif fpt.GreenesResidue > 0:
        marker = 'o'
    else:
        marker = 's'
    
    ax.scatter(fpt.x[0], fpt.z[0], c=colors[i+1], marker=marker, edgecolors='black', linewidths=1)

ax.set_aspect('equal')

#### Poincare plotter

In [5]:
from scipy.integrate import solve_ivp

## for now 2d startpoint in the phi = 0 plane
nfieldlines = 10
Rs = np.linspace(ma.gamma()[0, 0], ma.gamma()[0, 0] + 0.01, nfieldlines)
Zs = [ma.gamma()[0, 2] for i in range(nfieldlines)]
RZstart = np.array([[r, z] for r, z in zip(Rs, Zs)])
#RZstart = np.array([[1.6, 0.0],[1.6, 0.0]])

phis = [0]

In [6]:
RZstart

array([[1.59901829, 0.        ],
       [1.6001294 , 0.        ],
       [1.60124051, 0.        ],
       [1.60235162, 0.        ],
       [1.60346273, 0.        ],
       [1.60457384, 0.        ],
       [1.60568495, 0.        ],
       [1.60679607, 0.        ],
       [1.60790718, 0.        ],
       [1.60901829, 0.        ]])

In [7]:
import matplotlib.pyplot as plt

class PoincarePlanes():

    @classmethod
    def from_ivp(cls, out):
        cls.out = out

    @property
    def traces(self):
        RZs = self.out.y
        phis = self.out.t

        traces = np.array([RZs[:,i::2] for i in range(len(phis))])

        return [traces[2*i:2*i+2,:] for i in range (traces.shape[0]//2)]
        
    def plot(self):
        
        fig, ax = plt.subplots()
        for trace in self.traces:
            R, Z = trace
            plt.plot(R, Z, 'o', markersize=2)

        return fig, ax

In [9]:
def inv_Jacobian(R, phi, _):
    return np.array([
        [np.cos(phi), -np.sin(phi)/R, 0], 
        [np.sin(phi), np.cos(phi)/R, 0], 
        [0,0,1]
    ])

def poincare_ivp(bs, RZstart, phis, **kwargs):
    options = {
        "rtol": 1e-10,
        "atol": 1e-10,
        "nintersect": 10,
        "method": "DOP853",
    }
    options.update(kwargs)
    print(options)

    def Bfield_2D(t, rzs):
        print(rzs)
        rzs = rzs.reshape((-1, 2))
        rphizs = np.ascontiguousarray(np.vstack((rzs[:, 0], t*np.ones(rzs.shape[0]), rzs[:, 1])).T)
        bs.set_points_cyl(rphizs)
        Bs = list()
        for position, B in zip(rphizs, bs.B()):
            B = inv_Jacobian(*position) @ B.reshape(3, -1)
            Bs.append(np.array([B[0, 0] / B[1, 0], B[2, 0] / B[1, 0]]))

        return np.array(Bs).flatten()
    
    # setup the phis of the poincare sections
    phis = np.unique(np.mod(phis, 2*np.pi))
    phis.sort()

    # setup the evaluation points for those sections
    phi_evals = np.array([phis+2*np.pi*i for i in range(options['nintersect']+1)])
    
    out = solve_ivp(
        Bfield_2D,
        [0, phi_evals[-1,-1]],
        RZstart.flatten(),
        t_eval=phi_evals.flatten(),
        method=options["method"],
        rtol=options["rtol"],
        atol=options["atol"],
    )

    return out

In [1]:
out = poincare_ivp(bsh, RZstart, phis, rtol=1e-5, atol=1e-5, nintersect=10, method="DOP853")

NameError: name 'poincare_ivp' is not defined

In [ ]:
sections = PoincarePlanes.from_ivp(out)

In [ ]:
def B(rphiz):
    bsh.set_points_cyl(rphiz)
    return bsh.B().reshape(3, -1)

%timeit B(np.array([RZstart[:,0], np.zeros(RZstart.shape[0]),RZstart[:,1]], dtype=np.float64).reshape(-1,3))